In [1]:
from collections import OrderedDict
from contextlib import contextmanager
import random

# TODO(eric.cousineau): Use tensorboard in notebook.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader

In [2]:
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 1000

In [3]:
def zero_grad(p):
    if p.grad is not None:
        p.grad.zero_()

In [4]:
def l1_loss(y, yh):
    return torch.mean(torch.abs(y - yh))

def mse_loss(y, yh):
    return torch.mean((y - yh)**2)

In [5]:
def flat_cat_detached(ps):
    return torch.cat([p.detach().view(-1) for p in ps])

In [6]:
class SequentialDict(nn.Sequential):
    """
    We must use OrderedDict because otherwise pytorch will sort the keys... I think?
    See:
        https://discuss.pytorch.org/t/append-for-nn-sequential-or-directly-converting-nn-modulelist-to-nn-sequential/7104/4
        https://github.com/pytorch/pytorch/pull/40905
    """
    def __init__(self, *args, **kwargs):
        super().__init__(OrderedDict(*args, **kwargs))

In [7]:
def seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)

In [8]:
class MLP(nn.Module):
    """Simple Multi-Layer Perceptron."""
    def __init__(
        self,
        num_inputs,
        num_outputs,
        *,
        num_hidden_units,
        num_layers,
    ):
        super().__init__()
        assert num_layers >= 2
        self.layers = SequentialDict()
        self.layers.input = SequentialDict(
            fcn=nn.Linear(num_inputs, num_hidden_units),
            activation=nn.ReLU(),
        )
        hidden = []
        for i in range(num_layers - 2):
            hidden.append(SequentialDict(
                fcn=nn.Linear(num_hidden_units, num_hidden_units),
                activation=nn.ReLU(),
            ))
        self.layers.hidden = nn.Sequential(*hidden)
        self.layers.output = SequentialDict(
            fcn=nn.Linear(num_hidden_units, num_outputs)
        )

    def forward(self, x):
        return self.layers(x)

In [9]:
# Derived from this kinda goal: https://towardsdatascience.com/how-to-visualize-convolutional-features-in-40-lines-of-code-70b7d87b0030
class SaveActivations:
    def __init__(self):
        self.y = None

    def forward_hook(self, module, x, y):
        self.y = y.detach()

@contextmanager
def save_activations(module):
    savers = []
    hooks = []
    for m in module.modules():
        if isinstance(m, nn.ReLU):
            saver = SaveActivations()
            hook = m.register_forward_hook(saver.forward_hook)
            savers.append(saver)
            hooks.append(hook)
    assert len(hooks) > 0
    yield savers
    for hook in hooks:
        hook.remove()

def compute_activation_ratios(savers, clear=True):
    activation_ratios = []
    for saver in savers:
        assert saver.y is not None
        activation_ratios.append((saver.y > 0).to(torch.float).mean())
        if clear:
            saver.y = None
    return torch.tensor(activation_ratios)

In [10]:
@torch.no_grad()
def shit_init_param(p, scale=1.0, offset=1e-5):
    values = torch.linspace(-1.0, 1.0, p.numel()) / p.numel()
    sign = torch.sign(values)
    sign[sign == 0] = 1.0
    values += offset * sign
    p[:] = scale * values.reshape(p.shape)

def shit_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            shit_init_param(m.weight)
            shit_init_param(m.bias, scale=0.1)

In [11]:
# Parameters.
amplitude = 1.0
period_sec = 1.0
shift_sec = 0.0
num_periods = 1.0
dt = 0.1
count_per_period = int(np.ceil(period_sec / dt))
count = num_periods * count_per_period
t = torch.arange(count) * dt

# def waveform(t):
#     omega = 2 * np.pi / period_sec
#     x = omega * (t + shift_sec)
#     return amplitude * torch.sin(x)

def waveform(t):
    c = 2.0
    return c * t

In [12]:
def rel_mean_abs(x, tol=torch.tensor(1e-8)):
    xa = x.abs()
    xa_div = torch.fmax(xa.max(), tol)
    xa_mean = xa.mean()
    return xa_mean / xa_div

def mean_abs(x):
    return x.abs().mean()

In [13]:
def fit(model, loss_fn, lr, t, num_epochs=3, batch_size=1):
    model.train()
    # Expected param + labeled dataset.
    t = t.unsqueeze(-1)
    y = waveform(t)

    dataset = list(zip(t, y))
    loader = DataLoader(dataset, batch_size=batch_size)

    # Logging
    dfs = []

    with save_activations(model) as savers:
        
        def simple_log(epoch, batch_idx):
            # Show err.
            activation_ratios = compute_activation_ratios(savers).numpy()
            activation_ratios_str = ", ".join([f"{x:.2f}" for x in activation_ratios])
            dp = p - p_prev
            dfs.append(pd.DataFrame(
                {
                    "epoch": epoch,
                    "batch_idx": batch_idx,
                    "mean|p|": mean_abs(p).detach().numpy(),
                    "mean|Δp|": mean_abs(dp).numpy(),
                    "loss": loss.detach().numpy(),
                    "activation_ratios": activation_ratios_str,
                },
                index=[len(dfs)],
            ))

        params = list(model.parameters())  # this is iter
        p = flat_cat_detached(params)
        p_prev = p.clone()
        with torch.no_grad():
            yh0 = model(t)
            loss = loss_fn(y, yh0)
        simple_log(epoch="pre-opt", batch_idx="n/a")

        for epoch in range(num_epochs):
            for batch_idx, (ti, yi) in enumerate(loader):
                zero_grad(p)
                yhi = model(ti)
                loss = loss_fn(yi, yhi)
                loss.backward()

                # SGD update, no momentum.
                with torch.no_grad():
                    for param in params:
                        v = param.grad  # velocity
                        param -= lr * v  # step

                # Validation.
                p = flat_cat_detached(params)
                simple_log(epoch, batch_idx)
                p_prev = p

    print(loss_fn)
    display(pd.concat(dfs))

In [14]:
def make_model():
    model = MLP(1, 1, num_hidden_units=20, num_layers=20)
    shit_init(model)
    return model

In [15]:
# Merp
fit(make_model(), loss_fn=mse_loss, lr=0.2, t=t, num_epochs=10, batch_size=len(t))

<function mse_loss at 0x7f213b9e2b70>


,epoch,batch_idx,mean|p|,mean|Δp|,loss,activation_ratios
0,pre-opt,n/a,0.001476,0.000000,1.328135,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
1,0,0,0.001518,0.000068,1.328135,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
2,1,0,0.001631,0.000113,0.684703,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
3,2,0,0.001739,0.000108,0.332519,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
4,3,0,0.001782,0.000067,0.788467,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
5,4,0,0.001740,0.000054,1.372087,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
6,5,0,0.001745,0.000125,1.209236,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
7,6,0,0.001751,0.000170,0.550818,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
8,7,0,0.001749,0.000144,0.371571,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
9,8,0,0.001693,0.000152,0.945545,"0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50"
